## Imports

In [4]:
import sys
import os
repo_path = os.path.abspath('../')
sys.path.append(repo_path)
import scripts.addSA2 as addSA2
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import geoplot.crs as gcrs
import geopandas as gpd
import geoplot as gplt
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import f_regression, SelectKBest


from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

import statistics as stat
from sklearn.model_selection import cross_val_score

## Data Download

The first thing we need to do is download the datasources needed for our analysis.

This involves:
-> median income by SA2 level
-> population by SA2 level
-> SA2 Shape files
-> school location data
-> postcode/sa2 data
(see data_download.py in the script folder for details)

In [3]:
%run ../scripts/data_download.py

INFO:root:Beginning zone download
INFO:root:Finished zone download
replace ../data/raw/abs_data/zone_data/SA2_2021_AUST_GDA2020.dbf? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)
INFO:root:Beginning population download


Archive:  ../data/raw/abs_data/zone_data/zones.zip


INFO:root:Finished population download
INFO:root:Beginning income download
INFO:root:Finished population download
INFO:root:Beginning population projection download
INFO:root:Finished population projection download
INFO:root:Beginning school download
INFO:root:Finished school download
INFO:root:Beginning postcode download
INFO:root:Finished postcode download


## Scraping

For this project we decided to scrape rental data from domain.com.au.
This website has a huge number of listings, and by searching by postcode we collected a large number of metrics from each listed property.

This process is done in scrape.ipynb, and the results are saved in the raw data folder. as scraped_properties.csv as seen below.
(This will not be run in this notebook as it takes an extremely long time)

In [4]:
scraped = pd.read_csv(f"../data/raw/scraped_properties.csv")
scraped.head(3)

,url,price,address,num_beds,num_bath,num_car,property_type,agent,bond,internal_area,...,performance_sold_this_year,performance_avg_days_on_market,demographic_population,demographic_average_age,demographic_owner,demographic_renter,demographic_family,demographic_single,latitude,longitude
0,https://www.domain.com.au/40-esmond-street-ard...,$400,40 Esmond Street Ardeer VIC 3022,3 Beds,2 Baths,1 Parking,House,Create Vic Real Estate,Bond $1738,NaN,...,Sold this year: 37,Avg days on market: 44,"Population: 3,099",Average age: 20 to 39,67%,33%,41%,59%,-37.775134,144.797067
1,https://www.domain.com.au/11-grant-avenue-gisb...,$600.00,11 Grant Avenue Gisborne VIC 3437,4 Beds,2 Baths,1 Parking,House,Brad Teal Woodards Gisborne,Bond $2608,NaN,...,Sold this year: 92,Avg days on market: 41,"Population: 9,000",Average age: 40 to 59,83%,17%,58%,42%,-37.493363,144.595569
2,https://www.domain.com.au/15-balmoral-avenue-b...,$575,15 Balmoral Avenue Bundoora VIC 3083,3 Beds,1 Bath,3 Parking,House,Thomas Quixley Fairfield Pty Ltd,Bond $2499,NaN,...,Sold this year: 229,Avg days on market: 52,"Population: 28,681",Average age: 20 to 39,72%,28%,47%,53%,-37.707414,145.065424


## API Data

Now we get a bunch of distance data for various properties that we can merge into our other data

This is done with the school_proximities.ipynb notebook. But is quite an involved process, so that data is saved inside the git repository instead of being run here.

## Pre-Processing

While this gives us a lot of raw data, it needs to be put into a useable format first. This is done in pre_processing.py. By using a variety of regex rules to extract the relevant numbers from the data. We also need to remove a specific NSW town that was caught in the scrape for some reason. We also remove carspaces as they are not relevant to the problems.

From there we merge with the API distance data that has been gathered.

Now we can remove outliers. This is done mainly by plotting a linear regression model using:
-> number of beds
-> number of baths
-> number of parking spaces
-> distance to a school
-> distance to a park
-> distance to a shop
-> bond
and use this to predict rent price.

From this we reject values with a high cooks distance.
(See pre_processing.ipynb for proper details, as the .py file is purely for this notebook)

In [1]:
%run ../scripts/pre_processing.py

Instances before outlier removal: 15237
Instances after  outlier removal: 13416


,count,weekly_rent,bond,num_beds,num_baths,num_parking,internal_area,land_area,other_sold_n_bed_suburb,neighbourhood_under_20,neighbourhood_20_to_39,neighbourhood_40_to_59,neighbourhood_above_60,neighbourhood_long_term_residents,neighbourhood_owners,neighbourhood_renter,neighbourhood_family,neighbourhood_single,performance_median_price,performance_auction_clearance,performance_sold_this_year,performance_avg_days_on_market,demographic_population,demographic_owner,demographic_renter,demographic_family,demographic_single,school_distance,park_distance,shop_distance,train_distance,stop_distance,population,median_weekly_income
suburb,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
abbotsford,52,548.600000,2387.020000,2.000000,1.365385,1.195122,NaN,NaN,79.120000,0.107692,0.555577,0.230577,0.104231,0.399231,0.347600,0.652400,0.505000,0.495000,7.622340e+05,0.514255,79.916667,72.510638,8443.680000,0.396000,0.604000,0.259200,0.740800,772.030769,434.905882,1081.582609,1406.405769,398.075000,9780.153846,1323.153846
abeckett street,19,501.842105,2470.909091,1.421053,1.105263,1.000000,NaN,NaN,455.736842,0.063158,0.776842,0.131579,0.033684,0.180714,0.260714,0.739286,0.285714,0.714286,4.430714e+05,0.436429,463.357143,127.357143,44828.428571,0.307143,0.692857,0.242143,0.757857,707.547368,493.621053,773.422222,388.752632,321.731579,17545.368421,675.105263
ada,25,455.416667,2034.720000,2.680000,1.800000,1.500000,NaN,NaN,135.750000,0.209565,0.466522,0.210870,0.107826,0.573000,0.550435,0.449565,0.528261,0.471739,7.270476e+05,0.534706,135.750000,71.761905,13646.280000,0.550000,0.450000,0.391200,0.608800,1815.372000,4185.372000,6630.104762,3456.820000,537.276000,12763.040000,781.040000
airport west,17,428.529412,1909.142857,2.529412,1.294118,1.294118,NaN,299.0,48.588235,0.222353,0.261765,0.262941,0.251765,0.621765,0.721765,0.278235,0.545882,0.454118,7.813750e+05,0.704375,48.588235,70.928571,7567.000000,0.730000,0.270000,0.470000,0.530000,1007.452941,504.223529,1178.320000,7175.617647,260.870588,8237.000000,883.000000
albert park,17,900.882353,4799.375000,2.352941,1.529412,1.100000,NaN,179.0,47.470588,0.260000,0.205294,0.304706,0.227059,0.507647,0.618235,0.381765,0.595882,0.404118,1.839176e+06,0.751818,47.470588,36.454545,6210.000000,0.610000,0.390000,0.470000,0.530000,509.900000,364.858824,2331.300000,4022.794118,247.176471,16023.000000,1352.000000


Now our data looks like this:

In [2]:
pre_processed = pd.read_csv(f"../data/curated/pre_processed_data.csv")
pre_processed.head(3)

,url,postcode,suburb,state,weekly_rent,bond,num_beds,num_baths,num_parking,property_type,internal_area,land_area,last_sold,other_sold_n_bed_suburb,neighbourhood_under_20,neighbourhood_20_to_39,neighbourhood_40_to_59,neighbourhood_above_60,neighbourhood_long_term_residents,neighbourhood_owners,neighbourhood_renter,neighbourhood_family,neighbourhood_single,performance_median_price,performance_auction_clearance,performance_sold_this_year,performance_avg_days_on_market,demographic_population,demographic_owner,demographic_renter,demographic_family,demographic_single,demographic_average_age,latitude,longitude,school_duration,school_distance,park_duration,park_distance,shop_duration,shop_distance,train_duration,train_distance,stop_duration,stop_distance
0,https://www.domain.com.au/40-esmond-street-ardeer-vic-3022-16035224,3022,esmond,vic,400.0,1738.0,3.0,2.0,1.0,House,NaN,NaN,2018.0,37.0,0.26,0.26,0.26,0.21,0.66,0.74,0.26,0.49,0.51,680000.0,0.56,37.0,44.0,3099.0,0.67,0.33,0.41,0.59,20 to 39,-37.775134,144.797067,8.303333,5332.2,0.101667,42.6,5.595000,2903.6,8.285000,5710.8,1.300000,639.8
1,https://www.domain.com.au/11-grant-avenue-gisborne-vic-3437-16035741,3437,gisborne,vic,600.0,2608.0,4.0,2.0,1.0,House,NaN,NaN,2022.0,92.0,0.25,0.23,0.28,0.24,0.67,0.81,0.19,0.62,0.38,1098000.0,0.69,92.0,41.0,9000.0,0.83,0.17,0.58,0.42,40 to 59,-37.493363,144.595569,1.713333,717.6,16.075000,15921.0,17.856667,17363.3,8.270000,6053.8,1.345000,489.0
2,https://www.domain.com.au/15-balmoral-avenue-bundoora-vic-3083-16058638,3083,balmoral,vic,575.0,2499.0,3.0,1.0,3.0,House,NaN,NaN,2014.0,229.0,0.23,0.26,0.27,0.25,0.79,0.84,0.16,0.57,0.43,830000.0,0.75,229.0,52.0,28681.0,0.72,0.28,0.47,0.53,20 to 39,-37.707414,145.065424,0.858333,214.6,1.670000,423.2,7.698333,3781.9,5.788333,2812.4,1.083333,298.9


We can also add in the SA2 zone and corresponding geometry for each of these listings. This allows us to visualise the data in various ways.

In [34]:
sa2_grouped = addSA2.addSA2(pre_processed, use_postcode=False)
sa2_grouped = sa2_grouped.groupby("SA2").mean().reset_index()

shape = gpd.read_file('../data/raw/ShapeFile/SA2_2021_AUST_GDA2020.shp')
shape = shape.loc[shape.STE_NAME21 == "Victoria"]
shape = shape.loc[shape.geometry != None]
shape["SA2_CODE21"] = pd.to_numeric(shape["SA2_CODE21"], errors='ignore')
sa2_grouped = gpd.GeoDataFrame(sa2_grouped.join(shape.set_index("SA2_CODE21")["geometry"], on="SA2"))
sa2_grouped = sa2_grouped[sa2_grouped["geometry"] != None]

If you want to see a plot of any feature, simply type the feature into the "FEATURE HERE" section below:

In [ ]:
# Plot the lines for the visualisation
ax = gplt.polyplot(sa2_grouped, figsize=(100, 50))
# create a heatmap based on a particular feature
gplt.choropleth(
  sa2_grouped,
  hue="weekly_rent",
  edgecolor="black",
  linewidth=0.1,
  cmap="Greens",
  legend=True,
  ax=ax
)

## Question 1
Using the grouped suburb data we try to predict suburb's rent based on features previously gathered

### Read in data

In [5]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)

DIR_CUR = "../data/curated/"
DIR_PLT = "../plots/"

In [14]:
# read in suburb level version of data
df = pd.read_csv(f"{DIR_CUR}suburb_data.csv")
display(df.head(3))

,suburb,count,weekly_rent,bond,num_beds,num_baths,num_parking,internal_area,land_area,other_sold_n_bed_suburb,neighbourhood_under_20,neighbourhood_20_to_39,neighbourhood_40_to_59,neighbourhood_above_60,neighbourhood_long_term_residents,neighbourhood_owners,neighbourhood_renter,neighbourhood_family,neighbourhood_single,performance_median_price,performance_auction_clearance,performance_sold_this_year,performance_avg_days_on_market,demographic_population,demographic_owner,demographic_renter,demographic_family,demographic_single,school_distance,park_distance,shop_distance,train_distance,stop_distance,population,median_weekly_income
0,abbotsford,52,548.600000,2387.020000,2.000000,1.365385,1.195122,NaN,NaN,79.120000,0.107692,0.555577,0.230577,0.104231,0.399231,0.347600,0.652400,0.505000,0.495000,762234.042553,0.514255,79.916667,72.510638,8443.680000,0.396000,0.604000,0.259200,0.740800,772.030769,434.905882,1081.582609,1406.405769,398.075000,9780.153846,1323.153846
1,abeckett street,19,501.842105,2470.909091,1.421053,1.105263,1.000000,NaN,NaN,455.736842,0.063158,0.776842,0.131579,0.033684,0.180714,0.260714,0.739286,0.285714,0.714286,443071.428571,0.436429,463.357143,127.357143,44828.428571,0.307143,0.692857,0.242143,0.757857,707.547368,493.621053,773.422222,388.752632,321.731579,17545.368421,675.105263
2,ada,25,455.416667,2034.720000,2.680000,1.800000,1.500000,NaN,NaN,135.750000,0.209565,0.466522,0.210870,0.107826,0.573000,0.550435,0.449565,0.528261,0.471739,727047.619048,0.534706,135.750000,71.761905,13646.280000,0.550000,0.450000,0.391200,0.608800,1815.372000,4185.372000,6630.104762,3456.820000,537.276000,12763.040000,781.040000


## Finding Most important features in predicting Rent
Using the F-score we find the 3 features of the suburb data to have the strongest predictive power to the median rent.

In [16]:
df_clean = df.drop(["suburb", "bond", "internal_area", "land_area"], axis=1)
df_clean = df_clean.dropna().reset_index(drop=True)

K_BEST = 3

X = df_clean.drop(["weekly_rent"], axis=1)
y = df_clean["weekly_rent"]

selector = SelectKBest(f_regression, k=K_BEST)
selector.fit_transform(X, y)
features_index = selector.get_support(indices=True)
predictors = list(X.iloc[:,features_index].columns)

print("The most important features are:")
for predictor in predictors:
    print(f"* {predictor}")

The most important features are:
* performance_median_price
* school_distance
* median_weekly_income


## Plots VS the weekly rent

In [ ]:
def vs(x, y, x_lab, y_lab, title, x_trans=lambda x: x, y_trans=lambda x: x):
    """creates a scatter plot between x and y"""
    plt.scatter(x_trans(x), y_trans(y), color="#fad051")
    plt.title(title, size=20)
    plt.xlabel(x_lab, size=16)
    plt.ylabel(y_lab, size=16)
    plt.show()

vs(df["school_distance"], df["weekly_rent"], "School Distance (m)", "Weekly Rent (AUS)", "Weekly Rent vs School Distance")
vs(df["median_weekly_income"], df["weekly_rent"], "Weekly Income (AUS)", "Weekly Rent (AUS)", "Weekly Rent vs Weekly Income")
vs(df["performance_median_price"], df["weekly_rent"], "Price (AUS)", "Weekly Rent (AUS)", "Weekly Rent vs Price")

### Modeling using the 3 most important features
A Linear Regression and Multi-Layer Perceptron Regressor are fitted between rent and the 3 best features identified above 

In [11]:
response = "weekly_rent"
scoring = "r2"

FOLDS = 8

def test_model(model, X, y):
    """Applys an 8-cross fold validation finding the R^2 for each fold"""
    scores = cross_val_score(model, X, y, scoring=scoring, cv=FOLDS)
    print(f"Min R^2:    {min(scores):.3f}")
    print(f"Max R^2:    {max(scores):.3f}")
    print(f"Median R^2: {stat.median(scores):.3f}")
    print(f"Mean R^2:   {stat.mean(scores):.3f}")

response = "weekly_rent"
predictors = ["median_weekly_income", "performance_median_price"] 

df_na_out = df.dropna(subset=predictors)

df_na_out = df.dropna(subset=predictors + [response]).reset_index(drop=True)

trans = ColumnTransformer(transformers=[("scale", StandardScaler(), predictors)]) 
X = trans.fit_transform(df_na_out[predictors])
y = df_na_out["weekly_rent"]

print("Linear Regression:")
test_model(LinearRegression(), X, y)

print("\nMulti-Layer Perceptron Regressor:")
test_model(MLPRegressor(random_state=30027, max_iter=1000000, n_iter_no_change=15), X, y)

Linear Regression:
Min R^2:    0.273
Max R^2:    0.782
Median R^2: 0.720
Mean R^2:   0.632

Multi-Layer Perceptron Regressor:
Min R^2:    0.202
Max R^2:    0.900
Median R^2: 0.678
Mean R^2:   0.585


## Question 2

For this question we need historical data instead of current, data. This has been scraped from domain.com.au, and is pre-processed seperately. (See predict_future_preprocess.ipynb for details)

In [8]:
%run ../scripts/predict_future_preprocessing.py

In [10]:
# We read in the dataframe for predictions
df = pd.read_csv(f"../data/curated/future_prediction_data.csv")

In [ ]:
df

In [ ]:
# For the linear model, exclude 0 clearance rate and no new building locations.
df = df[df["clearance"] != 0]
df = df[df["new_dwellings_2019"] != 0]

In [11]:
X = df[["2019_n_sold", "suburb_population", "new_dwellings_2019", "non_residential_value_2019"]]
y = df["3_year_growth"]

In [ ]:
# Fit a linear model to the data
reg = LinearRegression().fit(X,y)
reg.score(X,y)

Clearly, using a linear regression on historical data is not a valid method, so we will have to try another method to predict future value.
Instead we will have to take a non-statistical approach, and use the suburb metrics that we have access to.

## Predictor Metrics:

Historical 3-year-growth: While a regression to the mean is fairly likely in many cases, a suburb having a high growth rate in previous years should still indicate that that is more likely to continue.

Current growth rate: Extrapolating the 2022 growth rate to a 3 year prediction is unlikely to be accurate, but it is once again a decent indicator.

New dwellings / population: If there is a high number of new dwellings relative to the population, then in theory the increased supply will lead to decreased house value.

non residential value: The theory behind using this metric, is that non-residential housing indicates an increase in local business. This should in theory increase the value of the area. This may be incorrect, as it might indicate a non-residential area, but not sure.

Clearance: A high clearance rate should indicate a high demand in the area, so in theory an increase in future prices.

Average days on market: A low number of days on the market, should indicate (like clearance), a high demand for houses in the area.

sold / population: If there are a high number of sales relative to the population it indicates market interest.

Unfortunately due to a lack of historical data for many areas, and many of these metrics, we will have to decide aribtrarily on the importance of each feature. I will then create a ranking of each feature and use this as the input for the algorithm.

https://propertyupdate.com.au/property-investment-melbourne/#is-it-the-right-time-to-get-into-the-melbourne-property-market

https://www.trilogyfunding.com.au/blog/7-key-market-indicators-every-property-investor-should-understand-april-2015/


https://www.mmj.com.au/resources/blog/5-key-market-indicators-every-property-investor-should-know/

In [14]:
df = pd.read_csv(f"../data/curated/future_prediction_data.csv")

# Get rankings for each feature
predict_df = pd.DataFrame()
predict_df["suburb"] = df["suburb"]
predict_df["avg_days_on_market"] = df["avg_days_on_market"].rank(ascending = False) # low is better
predict_df["3_year_growth"] = df["3_year_growth"].rank(ascending = True) # high is better
predict_df["2022_growth"] = df["2022_growth"].rank(ascending = True) # high is better
predict_df["sold/pop"] = (df["2022_n_sold"]/df["suburb_population"]).rank(ascending = True).fillna(0) # high is better
predict_df["dwellings/pop"] = (df["new_dwellings_2021"]/df["suburb_population"]).rank(ascending = False).fillna(0) # low is better
predict_df["non_residential_value"] = df["non_residential_value_2021"].rank(ascending = True) # higher is better
predict_df["clearance"] = df["clearance"].rank(ascending = True) # higher is better
predict_df["sum"] = predict_df.sum(axis=1)
predict_df["SA2"] = df["SA2"]

/tmp/ipykernel_76/2844795967.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  predict_df["sum"] = predict_df.sum(axis=1)


In [13]:
predict_df = predict_df.sort_values(by="sum", ascending=False)
predict_df.to_csv(f"../data/curated/prediction_results.csv")

In [15]:
predict_df

,suburb,avg_days_on_market,3_year_growth,2022_growth,sold/pop,dwellings/pop,non_residential_value,clearance,sum,SA2
0,abbotsford,67.0,717.0,598.0,426.0,471.0,1173.0,911.0,4363.0,206041506.0
1,aberfeldie,236.0,1084.0,1018.0,310.0,372.0,859.0,1018.0,4897.0,206031501.0
2,aintree,376.0,808.0,745.0,474.0,67.0,737.0,878.0,4085.0,208041194.0
3,aireys-inlet,392.0,1190.0,1184.0,637.5,0.0,363.0,436.5,4203.0,NaN
4,airport-west,118.0,716.0,646.0,637.5,0.0,363.0,1042.0,3522.5,NaN
...,...,...,...,...,...,...,...,...,...,...
1191,yarroweyah,907.0,322.5,280.0,0.0,0.0,363.0,436.5,2309.0,NaN
1192,yellingbo,907.0,322.5,280.0,0.0,0.0,363.0,436.5,2309.0,NaN
1193,yelta,907.0,322.5,280.0,0.0,0.0,363.0,436.5,2309.0,NaN
1194,yinnar-south,907.0,322.5,280.0,0.0,0.0,363.0,436.5,2309.0,NaN


We can then see the ranking of suburbs. Flinders, Blairgowrie, tootgarook, mccrae, and mornington take the top spots.

In [16]:
shape = gpd.read_file('../data/raw/ShapeFile/SA2_2021_AUST_GDA2020.shp')
shape = shape.loc[shape.STE_NAME21 == "Victoria"]
shape = shape.loc[shape.geometry != None]
shape["SA2_CODE21"] = pd.to_numeric(shape["SA2_CODE21"], errors='ignore')
plot_df = predict_df.join(shape[["SA2_CODE21", "geometry"]].set_index("SA2_CODE21"), on="SA2")
plot_df = plot_df[plot_df["geometry"] != None]

In [ ]:
from matplotlib.pyplot import figure
import geoplot.crs as gcrs


ax = gplt.polyplot(shape, figsize=(100, 50))#, projection=gcrs.AlbersEqualArea())

gplt.choropleth(
  gpd.GeoDataFrame(plot_df),
  hue="sum",
  edgecolor="black",
  linewidth=0.1,
  cmap="Greens",
  legend=True,
  ax=ax
)

This graphic shows the results per SA2 zone, but because they don't line up with the suburbs very well, so we get a lot of missing values.

## Question 3

We start off by reading the data. We'll be using the processed dataset with *ptv*, *parks*, *schools*, and *shopping centres* proximity addded. Note that we'll be using the average proximity here

In [4]:
import pandas as pd
from pyrosm import OSM, get_data
import ssl
from scipy.stats import percentileofscore
import numpy as np
import geopandas as gpd
import folium
import sys
sys.path.append("../scripts")
from liveability_helper import median_rent, score

property_df = pd.read_csv(
    "/Users/matthewrush/Documents/generic-real-estate-consulting-project-group-32/data/curated/pre_processed_data.csv",
    usecols=[
        "url",
        "postcode",
        "school_duration",
        "school_distance",
        "park_duration",
        "park_distance",
        "shop_duration",
        "shop_distance",
    ],
)

postcode_df = pd.read_csv(
    "../data/raw/postcode.csv",
    usecols=["postcode", "locality"],
)

api_df = pd.read_csv(
    "../data/curated/api_data.csv",
    usecols=[
        "url",
        "train_duration",
        "train_distance",
        "stop_duration",
        "stop_distance",
    ],
)

historical_df = pd.read_csv(
    "../data/curated/historical_sales.csv",
    usecols=["postcode", "n_sold", "median_rent"],
)

property_df = pd.merge(property_df, api_df)
property_df = property_df.groupby("postcode").mean().reset_index()


Now we find the median rent of each suburb using the scraped data

In [5]:
property_df["median_rent"] = property_df.apply(
    lambda x: median_rent(historical_df, x["postcode"]), axis=1
)
property_df = property_df.where(pd.notnull(property_df["median_rent"]), 0)

property_df = property_df[property_df["median_rent"] != 0]


/Users/matthewrush/Documents/generic-real-estate-consulting-project-group-32/notebooks/../scripts/liveability_helper.py:12: RuntimeWarning: invalid value encountered in double_scalars
  weighted_ave = np.nansum([x * y for x, y in zip(n_sold, median)]) / np.nansum(


Now we find the liveability scores of each suburb.

The score is calculated by retrieving the percentile value of each factors of each suburb relative to other suburbs. 

This means the lower the score, the better. We're giving each factors equal weights here since we don't have evidence that suggest we should weight it otherwise

The code is hidden since it is quite long and doesn't do anything too complicated. If interested, please view 

```
/scripts/liveability_helper.py
```

In [6]:
property_df["score"] = property_df.apply(
    lambda x: score(
        property_df,
        x["school_duration"],
        x["school_distance"],
        x["park_duration"],
        x["park_distance"],
        x["shop_duration"],
        x["shop_distance"],
        x["train_duration"],
        x["train_distance"],
        x["stop_duration"],
        x["stop_distance"],
        x["median_rent"],
    ),
    axis=1,
)

property_df = property_df.sort_values("score")
# property_df.to_csv("../data/curated/q3_live.csv")


And here we are, the top 5 most liveable suburbs:


In [8]:
property_df[["postcode", "median_rent"]].head()


,postcode,median_rent
2,3003.0,453.409091
46,3056.0,529.145234
114,3141.0,555.925926
48,3058.0,495.157431
58,3070.0,582.108062


Let's do some visualisation now. We can see that the top 5 suburbs fall mostly in inner sububs with good PTV, schools and park areas.

In [ ]:
shape = gpd.read_file("../data/raw/abs_data/zone_data/SA2_2021_AUST_GDA2020.shp")
shape = shape.loc[shape.STE_NAME21 == "Victoria"]
shape = shape.loc[shape.geometry != None]
shape["SA2_CODE21"] = pd.to_numeric(shape["SA2_CODE21"], errors="ignore")

map_data = pd.DataFrame(shape["SA2_NAME21"])
map_data["liveable"] = 0

map_data.loc[
    map_data["SA2_NAME21"].isin(
        [
            "Brunswick - South",
            "West Melbourne - Residential",
            "South Yarra - North",
            "Northcote",
            "Armadale",
        ]
    ),
    "liveable",
] = 1


gdf = gpd.GeoDataFrame(
    shape
)

geoJSON = gdf[['SA2_NAME21', 'geometry']].set_index('SA2_NAME21').to_json()
# geoJSON = gdf[['LocationID', 'geometry', 'zone_x']].drop_duplicates(
#     'LocationID').set_index('zone_x').to_json()

_map = folium.Map(location=[-37.840935,144.946457],
                  tiles="cartodbpositron", zoom_start=11)
# refer to the folium documentations on how to plot aggregated data.
_map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name="choropleth",
    data=map_data,
    columns=["SA2_NAME21", "liveable"],
    key_on="feature.id",
    fill_opacity=0.7,
    line_opacity=.1,
    fill_color='Blues',))
_map





To find the most liveable **AND** affordable suburbs, we have to add in a penalising factor in mediant rent. We also add a constant multiplier in the penalising factor as well. This factor is tuned such that a (what we deem) sufficient amount of top suburbs fall around Melbourne median rent.

Note that the multiplier should deduct a more expensive suburb's score more than a cheaper suburb's score.

Also note that the median rent we're working with is:

In [12]:
property_df.median_rent.median()

411.3474613273453

In [11]:
property_df["affordability_score"] = property_df.apply(
    lambda x: score(
        property_df,
        x["school_duration"],
        x["school_distance"],
        x["park_duration"],
        x["park_distance"],
        x["shop_duration"],
        x["shop_distance"],
        x["train_duration"],
        x["train_distance"],
        x["stop_duration"],
        x["stop_distance"],
        x["median_rent"],
        affordable=True,
    ),
    axis=1,
)

property_df = property_df.sort_values("affordability_score")
property_df[["postcode", "median_rent"]].head()
# property_df.to_csv("../data/curated/q3_live_afford.csv")


,postcode,median_rent
2,3003.0,453.409091
0,3000.0,429.475164
145,3174.0,369.279346
6,3011.0,433.937785
14,3021.0,340.828125


The top 5 median rent is noticably lower across the top 5, besides **West Melbourne** which tops the list. The suburb just does so well with all of our criterias.

Let's do some visualisation again

In [ ]:
map_data = pd.DataFrame(shape["SA2_NAME21"])
map_data["liveable"] = 0
map_data.loc[
    map_data["SA2_NAME21"].isin(
        [
            "Melbourne CBD - West",
            "Melbourne CBD - East",
            "West Melbourne - Residential",
            "Noble Park - East",
            "Footscray",
            "St Albans - South",
        ]
    ),
    "liveable",
] = 1

_map = folium.Map(location=[-37.840935,144.946457],
                  tiles="cartodbpositron", zoom_start=11)
# refer to the folium documentations on how to plot aggregated data.
_map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name="choropleth",
    data=map_data,
    columns=["SA2_NAME21", "liveable"],
    key_on="feature.id",
    fill_opacity=0.7,
    line_opacity=.1,
    fill_color='Blues',))
_map

Noticably some pressure outwards to the outer suburbs. This is the penalising factor in play here.

This is a good starting point. We recommend adding more factors - for example crime rates, amenities/nightlife, and potentially traffic data to improve this ranking system. 

The proximity scores are also based on available properties for rent on domain. To get a better picture of a suburb's profile - consider calculating proximity potentually on a grid-based pattern. This may allow you to get a better understanding of the suburb without having to calculate proximity for every single property (though that is also an option)
